## IBM Data Science Capstone Project
### The Battle of Neighborhoods

#### Business Problem: FooBar Cycles is looking to add a new location to the Seacoast NH area

Import required packages

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json 
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from re import sub
from decimal import Decimal
from sklearn import preprocessing

print('Libraries imported.')

Libraries imported.


Define fucntion to extract table data from BeautifulSoup object

In [2]:
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
    return rows

Scrape zip-codes.com website for all towns within 30 miles of Rollinsford, NH

In [3]:
URL = "https://www.zip-codes.com/zip-code-radius-finder.asp?zipmileslow=0&zipmileshigh=30&zip1=03869&submit=Search"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find("div", {"id" : "tableview"})
list_table = tableDataText(table)
df = pd.DataFrame(list_table[1:], columns=list_table[0])
df = df.drop('#', axis = 1)
df = df[:-1]
df.tail()

,Zip,City,County,St,Country,Distance,
98,04061,NORTH WATERBORO,YORK,ME,US,29.18 miles,
99,03810,ALTON BAY,BELKNAP,NH,US,29.38 miles,
100,04006,BIDDEFORD POOL,YORK,ME,US,29.53 miles,
101,04056,NEWFIELD,YORK,ME,US,29.56 miles,
102,04063,OCEAN PARK,YORK,ME,US,29.60 miles,


Define function to scrape income, population, and location data from zipatlas.com across multiple pages 

In [4]:
def getIncomeData(base_URL, num_pages):
    df_income = pd.DataFrame()
    i = 0
    while i<num_pages:
        if i == 0:
            URL = base_URL.format('')
        else:
            URL = base_URL.format(str(i+1) + ".")
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        table = soup.find('div', {'id' : 'ctl00_ContentPlaceHolder1_final_content'})
        income_table = tableDataText(table)
        df_income = df_income.append(pd.DataFrame(income_table[4:], columns=income_table[3]))
        i += 1
    return df_income[:-3]

Getting data for Maine and New Hampshire towns, and cleaning data

In [5]:
me_income = getIncomeData("http://zipatlas.com/us/me/zip-code-comparison/median-household-income.{}htm", 4)
nh_income = getIncomeData("http://zipatlas.com/us/nh/zip-code-comparison/median-household-income.{}htm", 3)
income_data = nh_income.append(me_income)
income_data[['Latitude', 'Longitude']] = income_data['Location'].str.split(',', expand = True)
income_data = income_data.drop(['#', 'Location', 'National Rank'], axis = 1)
income_data = income_data.rename(columns = {'Avg. Income/H/hold': 'Household Income'})
income_data.head()

,Zip Code,City,Population,Household Income,Latitude,Longitude
0,03087,"Windham,New Hampshire","10,709","$94,794.00",42.811092,-71.302688
1,03049,"Hollis,New Hampshire","7,017","$92,847.00",42.749626,-71.585401
2,03031,"Amherst,New Hampshire","10,812","$88,671.00",42.874854,-71.600512
3,03750,"Etna,New Hampshire",962,"$86,421.00",43.708188,-72.195047
4,03233,"Elkins,New Hampshire",234,"$85,247.00",43.438900,-71.952889


Joining original town data with above income and location data, keeping only relevant cities and columns, and converting lat/long data to numeric columns to work with folium package

In [6]:
df = pd.merge(df, income_data,how="inner", left_on="Zip", right_on = "Zip Code")
df = df.drop(['City_x', 'County', 'St', 'Country', 'Zip'], axis = 1)
df = df.rename(columns = {'City_y': 'City'})
df['Latitude'] = pd.to_numeric(df['Latitude'])
df['Longitude'] = pd.to_numeric(df['Longitude'])
df.head()

,Distance,,Zip Code,City,Population,Household Income,Latitude,Longitude
0,0.00 miles,,03869,"Rollinsford,New Hampshire","2,407","$48,380.00",43.219845,-70.843112
1,3.20 miles,,03878,"Somersworth,New Hampshire","11,483","$43,177.00",43.253476,-70.887944
2,3.52 miles,,03820,"Dover,New Hampshire","28,537","$44,493.00",43.187186,-70.902839
3,5.29 miles,,03908,"South Berwick,Maine","6,682","$53,294.00",43.230492,-70.752426
4,5.83 miles,,03901,"Berwick,Maine","6,338","$44,519.00",43.293960,-70.836455


Mapping all towns within 30 miles of Rolllinsford, NH

In [7]:
rollinsford = df[df['City'] == 'Rollinsford,New Hampshire']
map_rollinsford = folium.Map(location = [rollinsford['Latitude'], rollinsford['Longitude']], zoom_start = 9)

for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rollinsford)  

map_rollinsford

Foursquare Credentials (redacted for privacy)

In [73]:
CLIENT_ID_sec = '****' # your Foursquare ID
CLIENT_SECRET_sec = '****' # your Foursquare Secret
ACCESS_TOKEN_sec = '****' # your FourSquare Access Token
VERSION_sec = '****'
print('Your credentails:')
print('CLIENT_ID: It\'s a secret')
print('CLIENT_SECRET: This one is really a secret!' )

Your credentails:
CLIENT_ID: It's a secret
CLIENT_SECRET: This one is really a secret!


Setting default radius and limit for FourSquare API

In [9]:
radius = 5000
LIMIT = 500

Defining function to identify all nearby venues to a given city, and pull the category type from this information

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                             'City Latitude', 
                             'City Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

Obtaining data for all nearby venues for each city in the df

In [35]:
nearby_venues = getNearbyVenues(names = df['City'],
                                latitudes = df['Latitude'],
                                longitudes = df['Longitude'],
                                radius = 5000)
nearby_venues.shape

(2520, 7)

In [85]:
print('There are {} uniques categories.'.format(len(nearby_venues['Venue Category'].unique())))

There are 250 uniques categories.


Implementing one-hot encoding on venue data and calculating mean of this data - essentially calculating percentage of each venue type withing the give city

In [36]:
venue_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix = "", prefix_sep="")
venue_onehot['City'] = nearby_venues['City']

tmp = venue_onehot.pop('City')
venue_onehot.insert(0, 'City', tmp)

city_venues = venue_onehot.groupby('City').mean().reset_index()
city_venues.head()

,City,Accessories Store,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Warehouse Store,Water Park,Waterfall,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Acton,Maine",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alfred,Maine",0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Alton Bay,New Hampshire",0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Alton,New Hampshire",0.0,0.0,0.0,0.153846,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Barnstead,New Hampshire",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Grouping columns by type to reduce number of variables for classification algorithms. [Note: Done at authors discretion]

In [64]:
shopping_cols = ['Accessories Store', 'Antique Shop', 'Arts & Crafts Store', 'Big Box Store', 'Bookstore', 'Boutique', 
                 'Clothing Store', 'Comic Shop', 'Convenience Store', 'Cosmetics Shop', 'Department Store', 
                 'Electronics Store', 'Fabric Shop', 'Flea Market', 'Flower Shop', 'Furniture / Home Store', 'Garden Center', 
                 'Gas Station', 'Gift Shop', 'Gourmet Shop', 'Hardware Store', 'Hobby Shop', 'Kids Store', 
                 'Liquor Store', 'Market', 'Mattress Store', "Men's Store", 'Discount Store', 'Mobile Phone Shop', 
                 'Miscellaneous Shop', 'Motorcycle Shop', 'Music Store', 'Optical Shop', 'Other Repair Shop', 'Outlet Mall', 
                 'Paper / Office Supplies Store', 'Pet Store', 'Shipping Store', 'Shoe Store', 'Shopping Mall', 'Shopping Plaza',
                 'Smoke Shop', 'Supplement Shop', 'Thrift / Vintage Store', 'Warehouse Store', 
                 "Women's Store", 'Video Game Store', 'Video Store']

travel_cols = ['Airport', 'Airport Terminal', 'Rental Car Location', 'Toll Plaza', 'Train Station']

dining_cols = ['American Restaurant', 'Asian Restaurant', 'Bagel Shop', 'Bakery', 'BBQ Joint', 'Bar', 'Bistro','Breakfast Spot',
               'Brewery', 'Burger Joint', 'Butcher', 'Cafeteria', 'Café','Candy Store','Caribbean Restaurant', 'Chinese Restaurant',
               'Cocktail Bar', 'Coffee Shop', 'College Cafeteria', 'Deli / Bodega', 'Dessert Shop', 'Diner', 'Donut Shop', 
               'Falafel Restaurant', 'Fast Food Restaurant', 'Fish Market', 'Food', 'Food & Drink Shop', 'Food Court',
               'Food Truck', 'French Restaurant', 'Fried Chicken Joint', 'Frozen Yogurt Shop', 'Chocolate Shop', 'Farmers Market', 
               'Gastropub', 'Greek Restaurant', 'Grocery Store', 'Hawaiian Restaurant', 'Indian Restaurant', 'Italian Restaurant',
               'Japanese Restaurant', 'Juice Bar', 'Mediterranean Restaurant', 'Mexican Restaurant', 
               'New American Restaurant', 'Pub', 'Restaurant', 'Hot Dog Joint', 'Ice Cream Shop', 'Pizza Place', 'Sandwich Place',
               'Seafood Restaurant','Snack Place', 'Soup Place', 'Steakhouse', 'Supermarket', 
               'Sushi Restaurant', 'Taco Place', 'Tea Room', 'Tex-Mex Restaurant', 'Thai Restaurant', 
               'Vegetarian / Vegan Restaurant', 'Wine Shop', 'Wings Joint']

fitness_cols = ['Athletics & Sports', 'Baseball Field', 'Basketball Court', 'Board Shop', 'Golf Course', 
                'Gym', 'Gym / Fitness Center', 'Gymnastics Gym', 'Skating Rink', 
                'Soccer Field', 'Sporting Goods Shop', 'Weight Loss Center']

entertainment_cols = ['Arcade', 'Beer Store',  'Bowling Alley',
                      'Dog Run', 'College Library', 'Disc Golf', 'Event Space', 'Gay Bar', 
                      'Gun Range', 'Harbor / Marina', 'Historic Site', 'History Museum', 'Intersection', 'Lounge', 'Mini Golf', 
                      'Movie Theater', 'Museum', 'Music Venue', 'Performing Arts Venue', 
                      'Salon / Barbershop', 'Sports Bar', 'Summer Camp', 'Theater', 'Wine Bar', 
                      'Rock Club', 'Science Museum']

outdoor_cols = ['Boat Launch', 'Boat or Ferry', 'Farm', 'Garden','Lake', 'Lighthouse', 'Mountain', 'Park', 
                'Pier', 'Playground', 'River', 'Scenic Lookout', 'State / Provincial Park',
                'Surf Spot', 'Trail', 'Vineyard', 'Nature Preserve']

hospitality_cols = ['Bed & Breakfast', 'Campground', 'Hotel', 'Inn', 'Motel', 'RV Park', 'Resort']

other_cols = ['Automotive Shop', 'Bank', 'Beach', 'Border Crossing', 'Business Service', 
              'Construction & Landscaping', 'Home Service', 'Insurance Office', 'Massage Studio', 
              'Multiplex', 'Pharmacy', 'Post Office','Rest Area', 'Storage Facility']

col_types = [shopping_cols, travel_cols, dining_cols, fitness_cols, entertainment_cols, outdoor_cols, hospitality_cols, other_cols] 

Summing data on each of above categories for each city

In [65]:
categorized_venues = pd.DataFrame()
categorized_venues['City'] = city_venues['City']
categorized_venues['Shopping'] = city_venues[shopping_cols].sum(axis = 1)
categorized_venues['Travel'] = city_venues[travel_cols].sum(axis = 1)
categorized_venues['Dining'] = city_venues[dining_cols].sum(axis = 1)
categorized_venues['Fitness'] = city_venues[fitness_cols].sum(axis = 1)
categorized_venues['Entertainment'] = city_venues[entertainment_cols].sum(axis = 1)
categorized_venues['Outdoor'] = city_venues[outdoor_cols].sum(axis = 1)
categorized_venues['Hospitality'] = city_venues[hospitality_cols].sum(axis = 1)
categorized_venues['Other'] = city_venues[other_cols].sum(axis = 1)
categorized_venues.head()

,City,Shopping,Travel,Dining,Fitness,Entertainment,Outdoor,Hospitality,Other
0,"Acton,Maine",0.000000,0.0,0.333333,0.0,0.0,0.222222,0.111111,0.222222
1,"Alfred,Maine",0.076923,0.0,0.615385,0.0,0.0,0.153846,0.076923,0.076923
2,"Alton Bay,New Hampshire",0.000000,0.0,0.200000,0.0,0.4,0.400000,0.000000,0.000000
3,"Alton,New Hampshire",0.076923,0.0,0.615385,0.0,0.0,0.076923,0.076923,0.153846
4,"Barnstead,New Hampshire",0.375000,0.0,0.250000,0.0,0.0,0.125000,0.125000,0.125000


Merging venue data with population and household income data, and cleaning

In [66]:
df_info = pd.merge(df[['City', 'Population', 'Household Income']], categorized_venues,how="inner", on = 'City')
df_info['Population'] = df_info['Population'].str.replace(',', '').astype(int)
df_info['Household Income'].replace(to_replace='[^0-9]+', value='',inplace=True,regex=True)
df_info['Household Income'] = df_info['Household Income'].astype(float)/100
df_info.head()

,City,Population,Household Income,Shopping,Travel,Dining,Fitness,Entertainment,Outdoor,Hospitality,Other
0,"Rollinsford,New Hampshire",2407,48380.0,0.190000,0.010000,0.640000,0.030000,0.060000,0.020000,0.000000,0.050000
1,"Somersworth,New Hampshire",11483,43177.0,0.261364,0.022727,0.443182,0.045455,0.056818,0.011364,0.034091,0.102273
2,"Dover,New Hampshire",28537,44493.0,0.184615,0.000000,0.600000,0.046154,0.061538,0.046154,0.000000,0.015385
3,"South Berwick,Maine",6682,53294.0,0.095238,0.000000,0.619048,0.095238,0.000000,0.142857,0.000000,0.000000
4,"Berwick,Maine",6338,44519.0,0.117647,0.000000,0.647059,0.000000,0.117647,0.000000,0.000000,0.058824


Scaling data using MinMaxScaler so that population and income do not have overwheliming effects on clustering, running clustering algorithm and applying labels to cities

In [67]:
k_clusters = 4
df_cluster = df_info.copy()
df_cluster_scaled = df_cluster.drop('City', 1)
df_cluster_scaled = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df_cluster_scaled), columns = df_cluster.columns[1:])
kmeans = KMeans(n_clusters = k_clusters, random_state = 0).fit(df_cluster_scaled)
df_cluster_scaled.insert(0, 'City', df_cluster['City'])
df_cluster.insert(0, 'Cluster #', kmeans.labels_)
df_cluster.head()

,Cluster #,City,Population,Household Income,Shopping,Travel,Dining,Fitness,Entertainment,Outdoor,Hospitality,Other
0,1,"Rollinsford,New Hampshire",2407,48380.0,0.190000,0.010000,0.640000,0.030000,0.060000,0.020000,0.000000,0.050000
1,0,"Somersworth,New Hampshire",11483,43177.0,0.261364,0.022727,0.443182,0.045455,0.056818,0.011364,0.034091,0.102273
2,0,"Dover,New Hampshire",28537,44493.0,0.184615,0.000000,0.600000,0.046154,0.061538,0.046154,0.000000,0.015385
3,2,"South Berwick,Maine",6682,53294.0,0.095238,0.000000,0.619048,0.095238,0.000000,0.142857,0.000000,0.000000
4,1,"Berwick,Maine",6338,44519.0,0.117647,0.000000,0.647059,0.000000,0.117647,0.000000,0.000000,0.058824


Joining labeled data with location data to map clusters

In [68]:
df_mapping = pd.merge(df_cluster, df[['City', 'Latitude', 'Longitude']], how = 'left', on = 'City')
df_mapping.head()

,Cluster #,City,Population,Household Income,Shopping,Travel,Dining,Fitness,Entertainment,Outdoor,Hospitality,Other,Latitude,Longitude
0,1,"Rollinsford,New Hampshire",2407,48380.0,0.190000,0.010000,0.640000,0.030000,0.060000,0.020000,0.000000,0.050000,43.219845,-70.843112
1,0,"Somersworth,New Hampshire",11483,43177.0,0.261364,0.022727,0.443182,0.045455,0.056818,0.011364,0.034091,0.102273,43.253476,-70.887944
2,0,"Dover,New Hampshire",28537,44493.0,0.184615,0.000000,0.600000,0.046154,0.061538,0.046154,0.000000,0.015385,43.187186,-70.902839
3,2,"South Berwick,Maine",6682,53294.0,0.095238,0.000000,0.619048,0.095238,0.000000,0.142857,0.000000,0.000000,43.230492,-70.752426
4,1,"Berwick,Maine",6338,44519.0,0.117647,0.000000,0.647059,0.000000,0.117647,0.000000,0.000000,0.058824,43.293960,-70.836455


Plotting clusters by color on the map to get general sense of locations of each cluster

In [69]:
# create map
map_clusters = folium.Map(location = [rollinsford['Latitude'], rollinsford['Longitude']], zoom_start = 9)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_mapping['Latitude'], df_mapping['Longitude'], df_mapping['City'], df_mapping['Cluster #']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Grouping data by cluster label and taking mean of columns to identify trends within the clusters. It appears as though the clusters roughly fit into the following categories:

Cluster #0: High population, medium income, high density of dining venues, low density of outdoor venues <br>
Cluster #1: Medium population and income, low density of fitness venues <br>
Cluster #2: Medium population, high income, high density of shopping and fitness venues, low density of hospitality venues <br>
Cluster #3: Low population, medium-high income, low density of shopping and dining venues, very high density of outdoor venues

In [70]:
df_cluster.groupby(['Cluster #']).mean()

,Population,Household Income,Shopping,Travel,Dining,Fitness,Entertainment,Outdoor,Hospitality,Other
Cluster #,,,,,,,,,,
0,16695.642857,46251.214286,0.150186,0.012087,0.557545,0.057631,0.037307,0.033083,0.032240,0.076723
1,3924.218750,46363.500000,0.166590,0.001151,0.485970,0.018566,0.028701,0.079740,0.043507,0.134495
2,4390.227273,63551.590909,0.175593,0.009460,0.442655,0.119544,0.039510,0.067932,0.005602,0.102608
3,2533.625000,52172.875000,0.042014,0.000000,0.134722,0.073264,0.084722,0.494792,0.094097,0.045139


Based on the above findings, towns in cluster #2 seem to be the best fit. This cluster represents towns that have a medium population compared to the surrounding areas, a high income, and the highest proportion of fitness veneus, indicating that towns that fit into this cluster are receptive to fitness venues and that they can succeed in these locations. <br><br>

While the above cluster seems to make the most sense, we want to identify towns that are similar to others where fitness venues can be succesful, but where there may not already be fitness venues present. The below shows town data sorted by the lowest presence of fitness venues. Ideally we would want to put our new location in a town with a fairly high population, high household income, and a low density of fitness venues currently present. <br><br>

Coupling the below data with the above graph shows that the towns in the south-western portion of the towns in question are all of this cluster #2, which likely represents a good spot for a new location. Fremont and Danville both fit all of the above criteria and are likely a good spot to start looking to expand for our new location. FooBar Cycles Fremont has a nice ring to it, don't you think?

In [71]:
df_cluster0 = df_cluster[df_cluster['Cluster #'] == 2]
df_cluster0.sort_values(by='Fitness', ascending = True)

,Cluster #,City,Population,Household Income,Shopping,Travel,Dining,Fitness,Entertainment,Outdoor,Hospitality,Other
64,2,"Chester,New Hampshire",3792,68571.0,0.000000,0.000000,0.666667,0.000000,0.111111,0.111111,0.000000,0.000000
19,2,"New Castle,New Hampshire",1011,83708.0,0.060000,0.000000,0.660000,0.010000,0.080000,0.080000,0.030000,0.030000
39,2,"Hampton Falls,New Hampshire",1891,75513.0,0.277778,0.013889,0.472222,0.013889,0.000000,0.041667,0.027778,0.111111
46,2,"Fremont,New Hampshire",3522,62237.0,0.290323,0.000000,0.483871,0.032258,0.096774,0.000000,0.000000,0.064516
57,2,"Danville,New Hampshire",3896,58063.0,0.190476,0.000000,0.619048,0.047619,0.047619,0.000000,0.000000,0.047619
54,2,"Kingston,New Hampshire",5852,61522.0,0.133333,0.000000,0.533333,0.066667,0.000000,0.066667,0.000000,0.200000
32,2,"North Hampton,New Hampshire",4259,66696.0,0.202381,0.000000,0.464286,0.071429,0.011905,0.011905,0.023810,0.178571
73,2,"Plaistow,New Hampshire",7747,61707.0,0.257732,0.020619,0.453608,0.072165,0.061856,0.010309,0.000000,0.092784
5,2,"Eliot,Maine",5954,52606.0,0.692308,0.000000,0.192308,0.076923,0.000000,0.038462,0.000000,0.000000
26,2,"Stratham,New Hampshire",6355,76726.0,0.187500,0.020833,0.520833,0.083333,0.041667,0.041667,0.041667,0.062500
